# Selecting Data and creating Training and Test sets

In [1]:
#imports
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt

#estimators
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import linear_model

#model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

#cross validation
from sklearn.cross_validation import train_test_split

/Users/radghosh/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#import data
rawData = pd.read_csv('creditOne.csv')
rawData.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,REPAY_STATUS_SEPT,REPAY_STATUS_AUGUST,REPAY_STATUS_JULY,REPAY_STATUS_JUNE,...,BILL_AMOUNT_JUNE,BILL_AMOUNT_MAY,BILL_AMOUNT_APRIL,AMOUNT_PAID_SEPT,AMOUNT_PAID_AUGUST,AMOUNT_PAID_JULY,AMOUNT_PAID_JUNE,AMOUNT_PAID_MAY,AMOUNT_PAID_APRIL,DEFAULT
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


rawData.info()

# One Hot Encoding

In [3]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
# integer encode
label_encoder = LabelEncoder()
rawData["REPAY_STATUS_SEPT"] = label_encoder.fit_transform(rawData["REPAY_STATUS_SEPT"])
rawData["REPAY_STATUS_AUGUST"] = label_encoder.fit_transform(rawData["REPAY_STATUS_AUGUST"])
rawData["REPAY_STATUS_JULY"] = label_encoder.fit_transform(rawData["REPAY_STATUS_JULY"])
rawData["REPAY_STATUS_JUNE"] = label_encoder.fit_transform(rawData["REPAY_STATUS_JUNE"])
rawData["REPAY_STATUS_MAY"] = label_encoder.fit_transform(rawData["REPAY_STATUS_MAY"])
rawData["REPAY_STATUS_APRIL"] = label_encoder.fit_transform(rawData["REPAY_STATUS_APRIL"])
rawData["SEX"] = label_encoder.fit_transform(rawData["SEX"])
rawData["EDUCATION"] = label_encoder.fit_transform(rawData["EDUCATION"])
rawData["MARRIAGE"] = label_encoder.fit_transform(rawData["MARRIAGE"])
rawData["DEFAULT"] = label_encoder.fit_transform(rawData["DEFAULT"])

In [4]:
rawData.dtypes

ID                     int64
LIMIT_BAL              int64
SEX                    int64
EDUCATION              int64
MARRIAGE               int64
AGE                    int64
REPAY_STATUS_SEPT      int64
REPAY_STATUS_AUGUST    int64
REPAY_STATUS_JULY      int64
REPAY_STATUS_JUNE      int64
REPAY_STATUS_MAY       int64
REPAY_STATUS_APRIL     int64
BILL_AMOUNT_SEPT       int64
BILL_AMOUNT_AUGUST     int64
BILL_AMOUNT_JULY       int64
BILL_AMOUNT_JUNE       int64
BILL_AMOUNT_MAY        int64
BILL_AMOUNT_APRIL      int64
AMOUNT_PAID_SEPT       int64
AMOUNT_PAID_AUGUST     int64
AMOUNT_PAID_JULY       int64
AMOUNT_PAID_JUNE       int64
AMOUNT_PAID_MAY        int64
AMOUNT_PAID_APRIL      int64
DEFAULT                int64
dtype: object

In [5]:
rawData.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,REPAY_STATUS_SEPT,REPAY_STATUS_AUGUST,REPAY_STATUS_JULY,REPAY_STATUS_JUNE,...,BILL_AMOUNT_JUNE,BILL_AMOUNT_MAY,BILL_AMOUNT_APRIL,AMOUNT_PAID_SEPT,AMOUNT_PAID_AUGUST,AMOUNT_PAID_JULY,AMOUNT_PAID_JUNE,AMOUNT_PAID_MAY,AMOUNT_PAID_APRIL,DEFAULT
0,1,20000,1,2,1,24,4,4,1,1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,1,2,2,26,1,4,2,2,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,1,2,2,34,2,2,2,2,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,1,2,1,37,2,2,2,2,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,0,2,1,57,1,2,1,2,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


# Selecting Features vs Depdendent Variable

In [7]:
#features
features = rawData.iloc[:, 1:24]
print('Summary of feature sample')
features.head()


Summary of feature sample


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,REPAY_STATUS_SEPT,REPAY_STATUS_AUGUST,REPAY_STATUS_JULY,REPAY_STATUS_JUNE,REPAY_STATUS_MAY,...,BILL_AMOUNT_JULY,BILL_AMOUNT_JUNE,BILL_AMOUNT_MAY,BILL_AMOUNT_APRIL,AMOUNT_PAID_SEPT,AMOUNT_PAID_AUGUST,AMOUNT_PAID_JULY,AMOUNT_PAID_JUNE,AMOUNT_PAID_MAY,AMOUNT_PAID_APRIL
0,20000,1,2,1,24,4,4,1,1,0,...,689,0,0,0,0,689,0,0,0,0
1,120000,1,2,2,26,1,4,2,2,2,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,1,2,2,34,2,2,2,2,2,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,1,2,1,37,2,2,2,2,2,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,0,2,1,57,1,2,1,2,2,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679


In [20]:
#Create a copy of the Feature set for Dimensioality Reduction
featuresDM = features
featuresDM.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,REPAY_STATUS_SEPT,REPAY_STATUS_AUGUST,REPAY_STATUS_JULY,REPAY_STATUS_JUNE,REPAY_STATUS_MAY,...,BILL_AMOUNT_JULY,BILL_AMOUNT_JUNE,BILL_AMOUNT_MAY,BILL_AMOUNT_APRIL,AMOUNT_PAID_SEPT,AMOUNT_PAID_AUGUST,AMOUNT_PAID_JULY,AMOUNT_PAID_JUNE,AMOUNT_PAID_MAY,AMOUNT_PAID_APRIL
0,20000,1,2,1,24,4,4,1,1,0,...,689,0,0,0,0,689,0,0,0,0
1,120000,1,2,2,26,1,4,2,2,2,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,1,2,2,34,2,2,2,2,2,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,1,2,1,37,2,2,2,2,2,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,0,2,1,57,1,2,1,2,2,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679


In [9]:
#dependent variable
depVar = rawData['DEFAULT']
depVar[1:10]

1    1
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: DEFAULT, dtype: int64

# Cross Validation

In [10]:
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(features, depVar, test_size=0.25, random_state=0)
X_train.shape, X_test.shape

((22500, 23), (7500, 23))

<b>Scaling the data<b>

In [11]:
# Scaling using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Building Models

In [12]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('Cross Val Score for Logistic regression classifier: ', cross_val_score(logreg, X_train, y_train))
print('Accuracy of Logistic regression classifier on training set: {:.3f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.3f}'
     .format(logreg.score(X_test, y_test)))

Cross Val Score for Logistic regression classifier:  [0.8096     0.80293333 0.80653333]
Accuracy of Logistic regression classifier on training set: 0.807
Accuracy of Logistic regression classifier on test set: 0.814


In [13]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Cross Val Score for Decision Tree classifier: ', cross_val_score(clf, X_train, y_train))
print('Accuracy of Decision Tree classifier on training set: {:.3f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.3f}'
     .format(clf.score(X_test, y_test)))

Cross Val Score for Decision Tree classifier:  [0.72146667 0.7204     0.72586667]
Accuracy of Decision Tree classifier on training set: 1.000
Accuracy of Decision Tree classifier on test set: 0.732


In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Cross Val Score for K-NN classifier: ', cross_val_score(knn, X_train, y_train))
print('Accuracy of K-NN classifier on training set: {:.3f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.3f}'
     .format(knn.score(X_test, y_test)))

Cross Val Score for K-NN classifier:  [0.7956     0.7884     0.79293333]
Accuracy of K-NN classifier on training set: 0.843
Accuracy of K-NN classifier on test set: 0.790


In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Cross Val Score for LDA classifier: ', cross_val_score(lda, X_train, y_train))
print('Accuracy of LDA classifier on training set: {:.3f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.3f}'
     .format(lda.score(X_test, y_test)))

Cross Val Score for LDA classifier:  [0.81013333 0.804      0.80773333]
Accuracy of LDA classifier on training set: 0.807
Accuracy of LDA classifier on test set: 0.815


In [16]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Cross Val Score for GNB classifier: ', cross_val_score(gnb, X_train, y_train))
print('Accuracy of GNB classifier on training set: {:.3f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.3f}'
     .format(gnb.score(X_test, y_test)))

Cross Val Score for GNB classifier:  [0.57733333 0.52933333 0.59013333]
Accuracy of GNB classifier on training set: 0.567
Accuracy of GNB classifier on test set: 0.564


In [17]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Cross Val Score for SVM classifier: ', cross_val_score(svm, X_train, y_train))
print('Accuracy of SVM classifier on training set: {:.5f}'
     .format(svm.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.5f}'
     .format(svm.score(X_test, y_test)))

Cross Val Score for SVM classifier:  [0.8232     0.81613333 0.8148    ]
Accuracy of SVM classifier on training set: 0.82364
Accuracy of SVM classifier on test set: 0.82173


<b>The best accuracy was observed for the SVM classifier at 82.17%</b>

# Improving Model Performance

<b>Remove certain features and see if that affects the model predictions. The features were elimiated one by one until the best performance was observed by removing the following features.</b>

In [21]:
#Remove certain features and see if that affects the model predictions
featuresDM = featuresDM.drop(columns=['SEX'])
featuresDM = featuresDM.drop(columns=['MARRIAGE'])
featuresDM = featuresDM.drop(columns=['EDUCATION'])
featuresDM = featuresDM.drop(columns=['AGE'])
featuresDM = featuresDM.drop(columns=['LIMIT_BAL'])
featuresDM = featuresDM.drop(columns=['BILL_AMOUNT_SEPT'])
featuresDM = featuresDM.drop(columns=['BILL_AMOUNT_AUGUST'])
featuresDM = featuresDM.drop(columns=['BILL_AMOUNT_JULY'])
featuresDM = featuresDM.drop(columns=['BILL_AMOUNT_JUNE'])
featuresDM = featuresDM.drop(columns=['BILL_AMOUNT_MAY'])
featuresDM = featuresDM.drop(columns=['BILL_AMOUNT_APRIL'])
featuresDM = featuresDM.drop(columns=['AMOUNT_PAID_SEPT'])
featuresDM = featuresDM.drop(columns=['AMOUNT_PAID_AUGUST'])
featuresDM = featuresDM.drop(columns=['AMOUNT_PAID_JULY'])
featuresDM = featuresDM.drop(columns=['AMOUNT_PAID_JUNE'])
featuresDM = featuresDM.drop(columns=['AMOUNT_PAID_MAY'])
featuresDM = featuresDM.drop(columns=['AMOUNT_PAID_APRIL'])
featuresDM.head()

,REPAY_STATUS_SEPT,REPAY_STATUS_AUGUST,REPAY_STATUS_JULY,REPAY_STATUS_JUNE,REPAY_STATUS_MAY,REPAY_STATUS_APRIL
0,4,4,1,1,0,0
1,1,4,2,2,2,3
2,2,2,2,2,2,2
3,2,2,2,2,2,2
4,1,2,1,2,2,2


<b>The feature set now contains just the Repayment Statuses for the past months.</b>

<b>Redefine the dependent variable and repeat the cross validation and scaling</b>

In [34]:
#dependent variable remains the same
depVar = rawData['DEFAULT']

#Repeat Cross Validation Steps
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(featuresDM, depVar, test_size=0.25, random_state=0)

# Scaling using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

<b>Repeat the model building steps:</b>

In [35]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('Cross Val Score for Logistic regression classifier: ', cross_val_score(logreg, X_train, y_train))
print('Accuracy of Logistic regression classifier on training set: {:.3f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.3f}'
     .format(logreg.score(X_test, y_test)))

Cross Val Score for Logistic regression classifier:  [0.8072 0.802  0.8044]
Accuracy of Logistic regression classifier on training set: 0.805
Accuracy of Logistic regression classifier on test set: 0.813


In [29]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Cross Val Score for Decision Tree classifier: ', cross_val_score(clf, X_train, y_train))
print('Accuracy of Decision Tree classifier on training set: {:.3f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.3f}'
     .format(clf.score(X_test, y_test)))

Cross Val Score for Decision Tree classifier:  [0.814      0.8108     0.80826667]
Accuracy of Decision Tree classifier on training set: 0.831
Accuracy of Decision Tree classifier on test set: 0.822


In [30]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Cross Val Score for K-NN classifier: ', cross_val_score(knn, X_train, y_train))
print('Accuracy of K-NN classifier on training set: {:.3f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.3f}'
     .format(knn.score(X_test, y_test)))

Cross Val Score for K-NN classifier:  [0.81213333 0.80133333 0.8036    ]
Accuracy of K-NN classifier on training set: 0.816
Accuracy of K-NN classifier on test set: 0.808


In [31]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Cross Val Score for LDA classifier: ', cross_val_score(lda, X_train, y_train))
print('Accuracy of LDA classifier on training set: {:.3f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.3f}'
     .format(lda.score(X_test, y_test)))

Cross Val Score for LDA classifier:  [0.80893333 0.8032     0.80586667]
Accuracy of LDA classifier on training set: 0.806
Accuracy of LDA classifier on test set: 0.815


In [32]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Cross Val Score for GNB classifier: ', cross_val_score(gnb, X_train, y_train))
print('Accuracy of GNB classifier on training set: {:.3f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.3f}'
     .format(gnb.score(X_test, y_test)))

Cross Val Score for GNB classifier:  [0.80733333 0.8004     0.8052    ]
Accuracy of GNB classifier on training set: 0.804
Accuracy of GNB classifier on test set: 0.809


In [33]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Cross Val Score for SVM classifier: ', cross_val_score(svm, X_train, y_train))
print('Accuracy of SVM classifier on training set: {:.5f}'
     .format(svm.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.5f}'
     .format(svm.score(X_test, y_test)))

Cross Val Score for SVM classifier:  [0.82146667 0.81853333 0.81746667]
Accuracy of SVM classifier on training set: 0.82187
Accuracy of SVM classifier on test set: 0.82587


<b>Feature Elimination or Dimensionality Reduction</b>
<t>Eliminating the feature sets one by one showed that the performance of the best performing model, SVM can be boosted slightly from 82.17% to 82.58% if the model runs on the a feature set containing just the Repayment Status information. However this boost may also be a result of creating a much simpler data set by removing almost all the features. </t>